# Jupyter Notebook da Integradora 4
## Aluno: Thiago Medina
## Orientadora: Isabel H. Manssour

Abaixo, os imports necessário para a realização de todo o notebook

In [196]:
#1
import sys
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install unidecode
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install xlrd
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install ssl

import ssl
import tweepy as tw
import pandas as pd
import matplotlib
import nltk
from nltk.tokenize import TweetTokenizer
import string
import re
from unidecode import unidecode

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score


from sklearn import svm
from sklearn.naive_bayes import GaussianNB

import numpy as np

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You sh

    ERROR: Command errored out with exit status 1:
     command: /Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-install-v24m9vtx/ssl/setup.py'"'"'; __file__='"'"'/private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-install-v24m9vtx/ssl/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-pip-egg-info-6_wgx1v5
         cwd: /private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-install-v24m9vtx/ssl/
    Complete output (6 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/wx/c047tw591yz4cytxbj_26lv40000gp/T/pip-install-v24m9vtx/ssl/setup.py", line 33
        print 'looki

## O trabalho todo está separado em 3 partes:
### 1- Coleta dos tweets
### 2- Pré-processamento dos tweets.
### 3- Treinamento e teste por parte do algoritmo

Iniciamos com a coleta dos tweets:

Necessário possuir suas chaves próprias para a coleta de tweets.

In [197]:
#2
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

In [198]:
#3
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

public_tweets = api.home_timeline()

### Query para a coleta de tweets.

Altere a query de pesquisa (pode-se utilizar operadores lógicos). Altere também a quantidade de tweets que deseja coletar.

In [ ]:
#4
query = '(#netflix) -filter:retweets'
#query = '(#covid19) OR (#covid) OR (#coronavirus)  -filter:retweets'
#query = '(#educacao AND (-@mpsnet AND -@mpsnet2)) -filter:retweets'

cursor_tweets = tw.Cursor(api.search,
            q=query,lang = 'pt',tweet_mode='extended').items(300)

Utilizamos apenas a coluna com o texto dos tweets e a data de criação dos mesmos.

In [ ]:
#5
tweets_dict = {}
tweets_dict = tweets_dict.fromkeys(['created_at', 'full_text'])

In [ ]:
#6
for tweet in cursor_tweets:
    for key in tweets_dict.keys():
        try:
            twvalue = tweet._json[key]
            tweets_dict[key].append(twvalue)
        except KeyError:
            twvalue = ""
            if(tweets_dict[key] is None):
                tweets_dict[key] = [twvalue]
            else:
                tweets_dict[key].append(twvalue)
        except:
            tweets_dict[key] = [twvalue]
        #print("tweets_dict[key]: {} - tweet[key]: {}".format(tweets_dict[key],  twvalue))

Abaixo, listamos os tweets coletados.

In [ ]:
#7
tweets_dict

Adicionamos esses tweets a um DataFrame, para poder manipular os dados.

In [ ]:
#8
dfTweets = pd.DataFrame.from_dict(tweets_dict)

Mostramos os primeiros 5 dados do nosso DataFrame:


In [ ]:
#9
dfTweets.head()

E aqui, alteramos os nomes das colunas para português.

In [ ]:
#10
dfTweets.rename(columns={"full_text": "Texto", "created_at": "Data de criação"})

Por fim, adicionamos uma coluna chamada "Sentimento", aonde será colocada a anotação manual do sentimento de cada um desses tweets, e salvamos o DataFrame em um arquivo Excel, aonde é mais simples a manipulação.

In [ ]:
#11
dfTweets.insert(2, "Sentimento1", " ")
dfTweets.insert(3, "Sentimento2", " ")
dfTweets.insert(4, "SentimentoFinal", " ")
dfTweets.to_excel("/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TweetsColetadosNetflix.xlsx")

## A partir daqui, já deve ter sido realizada a anotação manual dos tweets!!

### Tendo feita a anotação manual dos tweets, que no caso desse trabalho foi feito por 3 pessoas, adicionamos a tabela excel em uma variavel chamada tweets.

In [183]:
#12
tweets= pd.read_excel('/Users/gustavoduarte/Desktop/TCC-analise-sentimentos-main/TabelaNetflix.xlsx', index_col=0,  engine='openpyxl') 

Abaixo, vemos como ficou nossa tabela.
Lembrando que tweets classificados como 0 são negativos, como 1 são positivos e como 2 são neutros.

In [184]:
#13
tweets

,full_text,Sentimento1,Sentimento1.1,SentimentoFinal
0,#Grupos de #WhatsApp #Ecuador #Amistad #Ligar ...,2,2,2
1,Comecei a assistir #Demolidor na #Netflix e tô...,1,1,1
2,"Pôster da 2º temporada de Love, Death &amp; Ro...",2,2,2
3,Alguem assistindo @quemmatousarabr no @Netflix...,2,0,2
4,"Ansiedade ativa, laça logo a quinta temporada ...",2,2,2
...,...,...,...,...
295,Las pelis y series que abandonan #Netflix hoy:...,2,2,2
296,ELA VOLTOU! Netflix faz boa ação e contrata Li...,2,2,2
297,"Como eu sentia falta de uma série assim, vicia...",1,1,1
298,"DATAS PARA A ÚLTIMA TEMPORADA DE ""LA CASA DE P...",2,2,2


## A seguir, iniciaremos o trabalho de pré-processamento dos tweets dos nossos datasets.

### O pré-processamento tem como objetivo deixar o texto presente nos tweets mais "limpo". Está sendo feito com apenas 1 dataset, apenas como exemplo, mas a técnica foi aplicada nos 3 datasets. 

Vamos utilizar um tweet como exemplo para que possa ser verificada o resultado de cada uma das tarefas de pré-processamento.

In [185]:
#14
exemploPreProcess = (tweets.at[57,'full_text'])
print (exemploPreProcess)

Apesar da história ter muitas reviravoltas e ser totalmente imprevisível, a 2ª temporada de #QuienMatoASara apresenta várias incongruências sem pés nem cabeça...

Espreita a #review: https://t.co/MYGUazSwpb
#seriesdatv #Netflix https://t.co/lR2ltdCTiP


Para a tokenização dos tweets (separar a frase em pequenos termos que façam sentido), utilizamos uma biblioteca específica para tweets. Ela auxilia na separação de emojis, hashtags, menções e outras questões próprias do Twitter.

In [186]:
#15
tweet_tokenize = TweetTokenizer()
tweet_tokenize.tokenize(exemploPreProcess)

['Apesar',
 'da',
 'história',
 'ter',
 'muitas',
 'reviravoltas',
 'e',
 'ser',
 'totalmente',
 'imprevisível',
 ',',
 'a',
 '2ª',
 'temporada',
 'de',
 '#QuienMatoASara',
 'apresenta',
 'várias',
 'incongruências',
 'sem',
 'pés',
 'nem',
 'cabeça',
 '...',
 'Espreita',
 'a',
 '#review',
 ':',
 'https://t.co/MYGUazSwpb',
 '#seriesdatv',
 '#Netflix',
 'https://t.co/lR2ltdCTiP']

A seguir, baixamos uma base de "stopwords" em português. Stopwords são palavras que não agregam sentido ao texto. Artigos e preposições são exemplos de stopwords.

In [187]:
#16
try:
     _create_unverified_https_context =     ssl._create_unverified_context
except AttributeError:
     pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('stopwords')
def RemoveStopWords(tweet):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    textInWords = []
    for word in tweet.lower().split():
        if (word not in stopwords) and (word.isdigit() == False): 
            textInWords.append(word)

    res = ' '.join(map(str, textInWords))
    return res

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gustavoduarte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Podemos ver abaixo como fica o tweet após a remoção de suas stopwords.

In [188]:
#17
RemoveStopWords(exemploPreProcess)

'apesar história ter muitas reviravoltas ser totalmente imprevisível, 2ª temporada #quienmatoasara apresenta várias incongruências pés cabeça... espreita #review: https://t.co/myguazswpb #seriesdatv #netflix https://t.co/lr2ltdctip'

Para fazer a stemização de cada tweet (deixar apenas a raiz(radical) de cada termo), utilizamos uma biblioteca própria que nos ajuda a fazer isso. Essa bibloteca é específica para português. 

In [189]:
#18
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     /Users/gustavoduarte/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [190]:
#19
#stemmer = nltk.stem.porter.PorterStemmer()
stemmer = nltk.stem.RSLPStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tweet):
    textInWords = []
    for word in tweet.lower().split():
        textInWords.append(stemmer.stem(word))
    res = ' '.join(map(str, textInWords))
    return res


Podemos ver abaixo como fica o tweet após a sua stemização.

In [191]:
#20
stem_tokens(exemploPreProcess)

'apes da histór ter muit reviravolt e ser total imprevisível, a 2ª tempor de #quienmatoas apresent vár incongru sem pé nem cabeça... espreit a #review: https://t.co/myguazswpb #seriesdatv #netflix https://t.co/lr2ltdctip'

Abaixo, temos uma função de "limpeza". Nela, são excluídas quaisquer caracteres que não são letras. Também são excluídos links, hashtags e emojis. Todas as letras são passadas para minúsculo. Além disso, são excluídos dígitos.

In [192]:
#21
def Limpa_dados(tweet): ##M MUDAR NOME DESSA FUNCAO
    # remove links, pontos, virgulas,ponto e virgulas dos tweets E ACENTOS
    tweet = unidecode(re.sub(r"http\S+", "", tweet).lower().replace('.','')
                      .replace(';','').replace('+','').replace(']','').replace('[','').replace('\'','').replace(';','').replace('%','').replace('-',' ').replace(',',' ')
                      .replace('\n','').replace('\"','').replace(')','').replace('(','').replace('$','').replace('?','')
                      .replace('_','').replace(':','').replace('!','').replace('|','').replace('/','').replace(')',''))
    tweet = unidecode(re.sub(r"@[A-Za-z0-9]+", "", tweet))
    tweet = unidecode(re.sub(r"\#[A-Za-z0-9]+", "", tweet))
    
    tweet = unidecode(re.sub(r"[0-9]+[A-Za-z0-9]+", "", tweet)) #Remove digitos
    tweet = tweet.replace('  ', ' ') #Corrige espaços feitos a mais.
    tweet = tweet.replace('[?]', ' ') #Corrige outros errinhos.
    return (tweet)

Abaixo, vemos o tweets após a sua "limpeza".

In [193]:
#22
Limpa_dados(exemploPreProcess)

'apesar da historia ter muitas reviravoltas e ser totalmente imprevisivel a temporada de apresenta varias incongruencias sem pes nem cabecaespreita a  '

Por fim, executamos as 3 funções de pré-processamento e adicionamos a uma nova tabela Excel.

In [194]:
#23
#PreProcessa todos os tweets:

qtdTweets = len(tweets)
for tweet in range (qtdTweets):
    tweets.at[tweet, 'full_text'] =  Limpa_dados(str(tweets.at[tweet, 'full_text']))
    tweets.at[tweet, 'full_text'] =  RemoveStopWords(str(tweets.at[tweet, 'full_text']))
    #tweets.at[tweet, 'full_text'] =  stem_tokens(str(tweets.at[tweet, 'full_text']))

In [195]:
#24
tweets.to_excel("TabelaNetflixPreProcessados.xlsx")

## Começaremos então o processo de treinamento por parte dos algoritmos.

Primeiramente, pegamos os dados de treinamento e teste. Aqui estaremos treinando o algoritmo com os dados de covid (700 tweets) e testando com os dados de covid (300 tweets). Os dados vem de tabelas diferentes e são diferentes.

In [171]:
dadosTreino = pd.read_excel('TabelaCovidTreinoPreProcessados.xlsx').fillna(' ')
dadosTeste = pd.read_excel('TabelaCovidTestePreProcessados.xlsx').fillna(' ')

FileNotFoundError: [Errno 2] No such file or directory: 'TabelaCovidTreinoPreProcessados.xlsx'

In [54]:
dadosTreino.head()

,full_text,Sentimento1,Sentimento2,SentimentoFinal
0,noticia hora vai vacinar neste feriado entao f...,2,2,2
1,importante pais expliquem importancia medidas ...,2,2,2
2,veiculos contam sistema mensagem som informaco...,2,2,2
3,beijo tempos crise sara arapiun ednei arapiun ...,2,2,2
4,despacho intimados presidente jair bolsonaro m...,2,2,2


In [55]:
dadosTeste.head()

,full_text,Sentimento1,Sentimento2,SentimentoFinal
0,assumiu risco tomar morrer pesquisas demonstra...,0,0,0
1,india imagens hospital gtb nova delhi varios p...,0,2,0
2,brasil numero suspeitas casos confirmados nume...,2,2,2
3,boletim check up saudeola veja acoes melhorar ...,2,2,2
4,catorze estados distrito federal mantem ocupac...,0,0,0


Vamos pegar os textos dos tweets de treinamento e teste e seus respectivos sentimentos.

In [56]:
tweetsParaTreino = dadosTreino['full_text'].values
classesParaTreino = dadosTreino['SentimentoFinal'].values
tweetsParaTeste = dadosTeste['full_text'].values
classesParaTeste = dadosTeste['SentimentoFinal'].values


Utilizamos a técnica Holdout, separando os dados em 70% para treinamento e 30% para teste.

In [57]:
qtdTweetsTreino = len(dadosTreino)
qtdTweetsTreino

700

In [58]:
qtdTweetsTeste = len(dadosTeste)
qtdTweetsTeste

300

Aqui, instanciamos o vetorizador dos tweets. Ele é responsável pela transformação dos dados textuais em um formato que o algoritmo entenda. É utilizada a técnica Bag Of Words, aonde as palavras viram colunas em uma tabela e cada tweet terá uma representação de 0's e 1's. 

In [59]:
# Instancia o objeto que faz a vetorização dos dados de texto:
tweet_tokenizer = TweetTokenizer() 
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

Abaixo, vetorizamos os dados de treinamento

In [60]:
vect_tweetsTreino = vectorizer.fit_transform(tweetsParaTreino) 

Finalmente, faremos o treinamento do algoritmo. Utilizamos o algoritmo Multinomial Naive Bayes, o SVM.

In [61]:
classificador = MultinomialNB()
classificador.fit(vect_tweetsTreino, classesParaTreino)  

classificadorSVM = svm.SVC(kernel='linear')
classificadorSVM.fit(vect_tweetsTreino, classesParaTreino)

classificadorLR = LogisticRegression(random_state=0).fit(vect_tweetsTreino, classesParaTreino)


Já tendo o algoritmo sido treinado, faremos a vetorização dos dados de teste.

In [62]:
vect_tweetsTeste = vectorizer.transform(tweetsParaTeste) 

Abaixo, fazemos a predição dos dados de teste por parte do algoritmo já treinado.

In [63]:
result = classificador.predict(vect_tweetsTeste)
resultSVM = classificadorSVM.predict(vect_tweetsTeste)
resultLR = classificadorLR.predict(vect_tweetsTeste)

### Agora já possuímos o algoritmo treinado e os dados para verificar o resultado nos testes. Vamos prosseguir vendo o resultado das métricas.

Vamos ver a acurácia primeiramente.

In [64]:
acc = accuracy_score(classesParaTeste, result) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTeste, resultSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTeste, resultLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

62.67% 
64.0% 
66.67% 


Vamos verificar a matriz de confusão agora. Ela tem o objetivo de ver em que classes o algoritmo conseguiu classificar melhor.

In [25]:
print (pd.crosstab(classesParaTeste, result, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTeste, resultSVM, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTeste, resultLR, rownames=['Real'], colnames=['Predito'], margins=True))


Predito    0   1    2  All
Real                      
0         80   6   14  100
1         18  23   18   59
2         43  13   85  141
All      141  42  117  300
Predito   0   1    2  All
Real                     
0        58   6   36  100
1        13  27   19   59
2        25   9  107  141
All      96  42  162  300
Predito    0   1    2  All
Real                      
0         68   4   28  100
1         10  26   23   59
2         27   8  106  141
All      105  38  157  300


Vamos verificar, por fim, as outras métricas.

In [26]:
print(classification_report(classesParaTeste, result, labels = [0,1,2]))

print(classification_report(classesParaTeste, resultSVM, labels = [0,1,2]))

print(classification_report(classesParaTeste, resultLR, labels = [0,1,2]))

             precision    recall  f1-score   support

          0       0.57      0.80      0.66       100
          1       0.55      0.39      0.46        59
          2       0.73      0.60      0.66       141

avg / total       0.64      0.63      0.62       300

             precision    recall  f1-score   support

          0       0.60      0.58      0.59       100
          1       0.64      0.46      0.53        59
          2       0.66      0.76      0.71       141

avg / total       0.64      0.64      0.63       300

             precision    recall  f1-score   support

          0       0.65      0.68      0.66       100
          1       0.68      0.44      0.54        59
          2       0.68      0.75      0.71       141

avg / total       0.67      0.67      0.66       300



## Agora, faremos o treinamento com os dados da tabela de assuntos diversos (700 tweets). A tabela de teste segue sendo a mesma.

Esse processo é semelhante ao feito anteriormente, então não há necessidade de explicar cada etapa.

In [27]:
dadosTreinoDiversos = pd.read_excel('TabelaDiversosPreProcessados.xlsx').fillna(' ')

In [28]:
dadosTreinoDiversos.head()

,full_text,Sentimento1,Sentimento2,SentimentoFinal
0,ante malestar coa falta dialogo medidas xunta ...,0,0,0
1,pra gastar podemos gastar pouco acesse,2,2,2
2,oposicao reforma tributaria desconhecimento lobby,0,2,2
3,importancia poupanca composicao setorial,1,1,1
4,"""pib governo"" errado",0,2,0


In [29]:
tweetsParaTreinoDiversos = dadosTreinoDiversos['full_text'].values
classesParaTreinoDiversos = dadosTreinoDiversos['SentimentoFinal'].values

In [30]:
qtdTweetsTreino = len(dadosTreinoDiversos)
qtdTweetsTreino

700

In [31]:
vect_tweetsTreinoDiversos = vectorizer.fit_transform(tweetsParaTreinoDiversos)  #treina com diversos

In [36]:
classificadorDiversos = MultinomialNB()
classificadorDiversos.fit(vect_tweetsTreinoDiversos, classesParaTreinoDiversos) 

classificadorSVMDiv = svm.SVC(kernel='linear')
classificadorSVMDiv.fit(vect_tweetsTreinoDiversos, classesParaTreinoDiversos)

classificadorLRDiv = LogisticRegression(random_state=0).fit(vect_tweetsTreinoDiversos, classesParaTreinoDiversos)

In [38]:
vect_tweetsTeste = vectorizer.transform(tweetsParaTeste) 

In [39]:
resultDiversos = classificadorDiversos.predict(vect_tweetsTeste)

resultDiversosSVM = classificadorSVMDiv.predict(vect_tweetsTeste)

resultDiversosLR = classificadorLRDiv.predict(vect_tweetsTeste)

In [40]:
acc = accuracy_score(classesParaTeste, resultDiversos) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTeste, resultDiversosSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTeste, resultDiversosLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

56.0% 
54.0% 
56.0% 


In [41]:
print (pd.crosstab(classesParaTeste, resultDiversos, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTeste, resultDiversosSVM, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTeste, resultDiversosLR, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0   1    2  All
Real                     
0        40  19   41  100
1         6  30   23   59
2        25  18   98  141
All      71  67  162  300
Predito   0   1    2  All
Real                     
0        32  14   54  100
1         8  28   23   59
2        19  20  102  141
All      59  62  179  300
Predito   0   1    2  All
Real                     
0        28  14   58  100
1         4  30   25   59
2        15  16  110  141
All      47  60  193  300


In [42]:
print(classification_report(classesParaTeste, resultDiversos, labels = [0,1,2]))

print(classification_report(classesParaTeste, resultDiversosSVM, labels = [0,1,2]))

print(classification_report(classesParaTeste, resultDiversosLR, labels = [0,1,2]))

             precision    recall  f1-score   support

          0       0.56      0.40      0.47       100
          1       0.45      0.51      0.48        59
          2       0.60      0.70      0.65       141

avg / total       0.56      0.56      0.55       300

             precision    recall  f1-score   support

          0       0.54      0.32      0.40       100
          1       0.45      0.47      0.46        59
          2       0.57      0.72      0.64       141

avg / total       0.54      0.54      0.52       300

             precision    recall  f1-score   support

          0       0.60      0.28      0.38       100
          1       0.50      0.51      0.50        59
          2       0.57      0.78      0.66       141

avg / total       0.56      0.56      0.54       300



## Agora faremos a classificação em tabelas binárias, aonde há apenas as classes positiva e negativa (1 e 0).

Para tal, foram feitas cópias dos 3 datasets e excluídos os tweets neutros nessas cópias. Assim, faremos o treinamento e teste novamente nestes datasets.
Este processo já foi explicado anteriormente.

In [43]:
dadosTreinoBin = pd.read_excel('TabelaCovidTreinoPreProcessadosBin.xlsx').fillna(' ')
dadosTesteBin = pd.read_excel('TabelaCovidTestePreProcessadosBin.xlsx').fillna(' ')

In [44]:
len(dadosTreinoBin)

402

In [45]:
len(dadosTesteBin)

159

In [46]:
tweetsParaTreinoBin = dadosTreinoBin['full_text'].values
classesParaTreinoBin = dadosTreinoBin['SentimentoFinal'].values
tweetsParaTesteBin = dadosTesteBin['full_text'].values
classesParaTesteBin = dadosTesteBin['SentimentoFinal'].values


In [47]:
# Instancia o objeto que faz a vetorização dos dados de texto:
tweet_tokenizer = TweetTokenizer() 
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

In [48]:
vect_tweetsTreinoBin = vectorizer.fit_transform(tweetsParaTreinoBin) 

In [49]:
classificadorBin = MultinomialNB()
classificadorBin.fit(vect_tweetsTreinoBin, classesParaTreinoBin)

classificadorSVMBin = svm.SVC(kernel='linear')
classificadorSVMBin.fit(vect_tweetsTreinoBin, classesParaTreinoBin)

classificadorLRBin = LogisticRegression(random_state=0).fit(vect_tweetsTreinoBin, classesParaTreinoBin)

In [50]:
vect_tweetsTesteBin = vectorizer.transform(tweetsParaTesteBin) 

In [51]:
resultBin = classificadorBin.predict(vect_tweetsTesteBin)

resultBinSVM = classificadorSVMBin.predict(vect_tweetsTesteBin)

resultBinLR = classificadorLRBin.predict(vect_tweetsTesteBin)

In [52]:
acc = accuracy_score(classesParaTesteBin, resultBin) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTesteBin, resultBinSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTesteBin, resultBinLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

81.13% 
79.25% 
78.62% 


In [65]:
print (pd.crosstab(classesParaTesteBin, resultBin, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteBin, resultBinSVM, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteBin, resultBinLR, rownames=['Real'], colnames=['Predito'], margins=True))

Predito    0   1  All
Real                 
0         94   6  100
1         24  35   59
All      118  41  159
Predito    0   1  All
Real                 
0         91   9  100
1         24  35   59
All      115  44  159
Predito    0   1  All
Real                 
0         95   5  100
1         29  30   59
All      124  35  159


In [66]:
print(classification_report(classesParaTesteBin, resultBin, labels = [0,1]))

print(classification_report(classesParaTesteBin, resultBinSVM, labels = [0,1]))

print(classification_report(classesParaTesteBin, resultBinLR, labels = [0,1]))

             precision    recall  f1-score   support

          0       0.80      0.94      0.86       100
          1       0.85      0.59      0.70        59

avg / total       0.82      0.81      0.80       159

             precision    recall  f1-score   support

          0       0.79      0.91      0.85       100
          1       0.80      0.59      0.68        59

avg / total       0.79      0.79      0.78       159

             precision    recall  f1-score   support

          0       0.77      0.95      0.85       100
          1       0.86      0.51      0.64        59

avg / total       0.80      0.79      0.77       159



#### Agora faremos o treinamento com o dataset de assuntos diversos sem os tweets neutros.

In [67]:
dadosTreinoDiversosBin = pd.read_excel('TabelaDiversosPreProcessadosBin.xlsx').fillna(' ')

In [68]:
tweetsParaTreinoDiversosBin = dadosTreinoDiversosBin['full_text'].values
classesParaTreinoDiversosBin = dadosTreinoDiversosBin['SentimentoFinal'].values

In [69]:
qtdTweetsTreinoBin = len(dadosTreinoDiversosBin)
qtdTweetsTreinoBin

307

In [70]:
vect_tweetsTreinoDiversosBin = vectorizer.fit_transform(tweetsParaTreinoDiversosBin)  #treina com diversos

In [71]:
classificadorDiversosBin = MultinomialNB()
classificadorDiversosBin.fit(vect_tweetsTreinoDiversosBin, classesParaTreinoDiversosBin)

classificadorDiversosSVMBin = svm.SVC(kernel='linear')
classificadorDiversosSVMBin.fit(vect_tweetsTreinoDiversosBin, classesParaTreinoDiversosBin)

classificadorDiversosLRBin = LogisticRegression(random_state=0).fit(vect_tweetsTreinoDiversosBin, classesParaTreinoDiversosBin)

In [72]:
vect_tweetsTesteBin = vectorizer.transform(tweetsParaTesteBin) 

In [73]:
resultDiversosBin = classificadorDiversosBin.predict(vect_tweetsTesteBin)

resultDiversosBinSVM = classificadorDiversosSVMBin.predict(vect_tweetsTesteBin)

resultDiversosBinLR = classificadorDiversosLRBin.predict(vect_tweetsTesteBin)

In [74]:
acc = accuracy_score(classesParaTesteBin, resultDiversosBin) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTesteBin, resultDiversosBinSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTesteBin, resultDiversosBinLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

74.21% 
73.58% 
74.21% 


In [75]:
print (pd.crosstab(classesParaTesteBin, resultDiversosBin, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteBin, resultDiversosBinSVM, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteBin, resultDiversosBinLR, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0   1  All
Real                
0        69  31  100
1        10  49   59
All      79  80  159
Predito   0   1  All
Real                
0        71  29  100
1        13  46   59
All      84  75  159
Predito   0   1  All
Real                
0        68  32  100
1         9  50   59
All      77  82  159


In [76]:
print(classification_report(classesParaTesteBin, resultDiversosBin, labels = [0,1]))

print(classification_report(classesParaTesteBin, resultDiversosBinSVM, labels = [0,1]))

print(classification_report(classesParaTesteBin, resultDiversosBinLR, labels = [0,1]))

             precision    recall  f1-score   support

          0       0.87      0.69      0.77       100
          1       0.61      0.83      0.71        59

avg / total       0.78      0.74      0.75       159

             precision    recall  f1-score   support

          0       0.85      0.71      0.77       100
          1       0.61      0.78      0.69        59

avg / total       0.76      0.74      0.74       159

             precision    recall  f1-score   support

          0       0.88      0.68      0.77       100
          1       0.61      0.85      0.71        59

avg / total       0.78      0.74      0.75       159



## Por fim, vamos validar os resultados realizando o treinamento com os tweets de assuntos diversos e o teste com os tweets #netflix

Esta etapa de treinamento/teste é como as anteriores, logo, não há necessidade de explicar cada passo.

In [77]:
dadosTreinoDiversos = pd.read_excel('TabelaDiversosPreProcessados.xlsx').fillna(' ')
dadosTesteNetflix = pd.read_excel('TabelaNetflixPreProcessados.xlsx').fillna(' ')

In [78]:
tweetsParaTreinoDiv = dadosTreinoDiversos['full_text'].values
classesParaTreinoDiv = dadosTreinoDiversos['SentimentoFinal'].values
tweetsParaTesteNetflix = dadosTesteNetflix['full_text'].values
classesParaTesteNetflix = dadosTesteNetflix['SentimentoFinal'].values

In [79]:
qtdTweetsTreino = len(dadosTreinoDiversos)
qtdTweetsTreino

700

In [80]:
qtdTweetsTeste = len(dadosTesteNetflix)
qtdTweetsTeste

300

In [81]:
# Instancia o objeto que faz a vetorização dos dados de texto:
tweet_tokenizer = TweetTokenizer() 
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

In [116]:
vect_tweetsTreinoDiv = vectorizer.fit_transform(tweetsParaTreinoDiv)



  (0, 47)	1
  (0, 2567)	1
  (0, 2521)	1
  (0, 1576)	2
  (0, 2574)	1


In [83]:
classificadorValid = MultinomialNB()
classificadorValid.fit(vect_tweetsTreinoDiv, classesParaTreinoDiv)

classificadorValidSVM = svm.SVC(kernel='linear')
classificadorValidSVM.fit(vect_tweetsTreinoDiv, classesParaTreinoDiv)

classificadorValidLR = LogisticRegression(random_state=0).fit(vect_tweetsTreinoDiv, classesParaTreinoDiv)

In [84]:
vect_tweetsTesteNetflix = vectorizer.transform(tweetsParaTesteNetflix) 

In [85]:
resultValid = classificadorValid.predict(vect_tweetsTesteNetflix)

resultValidSVM = classificadorValidSVM.predict(vect_tweetsTesteNetflix)

resultValidLR = classificadorValidLR.predict(vect_tweetsTesteNetflix)

In [86]:
acc = accuracy_score(classesParaTesteNetflix, resultValid) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTesteNetflix, resultValidSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTesteNetflix, resultValidLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

55.67% 
56.67% 
58.67% 


In [87]:
print (pd.crosstab(classesParaTesteNetflix, resultValid, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflix, resultValidSVM, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflix, resultValidLR, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0   1    2  All
Real                     
0        15   3   44   62
1         7   9   49   65
2        13  17  143  173
All      35  29  236  300
Predito   0   1    2  All
Real                     
0        11   0   51   62
1         2   8   55   65
2         6  16  151  173
All      19  24  257  300
Predito   0   1    2  All
Real                     
0         8   0   54   62
1         1   5   59   65
2         3   7  163  173
All      12  12  276  300


In [88]:
print(classification_report(classesParaTesteNetflix, resultValid, labels = [0,1,2]))

print(classification_report(classesParaTesteNetflix, resultValidSVM, labels = [0,1,2]))

print(classification_report(classesParaTesteNetflix, resultValidLR, labels = [0,1,2]))

             precision    recall  f1-score   support

          0       0.43      0.24      0.31        62
          1       0.31      0.14      0.19        65
          2       0.61      0.83      0.70       173

avg / total       0.51      0.56      0.51       300

             precision    recall  f1-score   support

          0       0.58      0.18      0.27        62
          1       0.33      0.12      0.18        65
          2       0.59      0.87      0.70       173

avg / total       0.53      0.57      0.50       300

             precision    recall  f1-score   support

          0       0.67      0.13      0.22        62
          1       0.42      0.08      0.13        65
          2       0.59      0.94      0.73       173

avg / total       0.57      0.59      0.49       300



Abaixo, o treinamento/teste aonde sem tweets neutros.

In [89]:
dadosTreinoDiversosBin = pd.read_excel('TabelaDiversosPreProcessadosBin.xlsx').fillna(' ')
dadosTesteNetflixBin = pd.read_excel('TabelaNetflixPreProcessadosBin.xlsx').fillna(' ')

In [90]:
tweetsParaTreinoDiversosBin = dadosTreinoDiversosBin['full_text'].values
classesParaTreinoDiversosBin = dadosTreinoDiversosBin['SentimentoFinal'].values
tweetsParaTesteNetflixBin = dadosTesteNetflixBin['full_text'].values
classesParaTesteNetflixBin = dadosTesteNetflixBin['SentimentoFinal'].values

In [91]:
# Instancia o objeto que faz a vetorização dos dados de texto:
tweet_tokenizer = TweetTokenizer() 
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

In [92]:
vect_tweetsTreinoBin = vectorizer.fit_transform(tweetsParaTreinoDiversosBin) 

In [93]:
classificadorBinValid = MultinomialNB()
classificadorBinValid.fit(vect_tweetsTreinoBin, classesParaTreinoDiversosBin)

classificadorBinValidSVM = svm.SVC(kernel='linear')
classificadorBinValidSVM.fit(vect_tweetsTreinoBin, classesParaTreinoDiversosBin)

classificadorBinValidLR = LogisticRegression(random_state=0).fit(vect_tweetsTreinoBin, classesParaTreinoDiversosBin)

In [94]:
vect_tweetsTesteNetflixBin = vectorizer.transform(tweetsParaTesteNetflixBin) 

In [99]:
resultValidBin = classificadorBinValid.predict(vect_tweetsTesteNetflixBin)

resultValidBinSVM = classificadorBinValidSVM.predict(vect_tweetsTesteNetflixBin)

resultValidBinLR = classificadorBinValidLR.predict(vect_tweetsTesteNetflixBin)

In [100]:
acc = accuracy_score(classesParaTesteNetflixBin, resultValidBin) * 100
print("{}{} ".format(acc.round(2), "%"))

acc2 = accuracy_score(classesParaTesteNetflixBin, resultValidBinSVM) * 100
print("{}{} ".format(acc2.round(2), "%"))

acc3 = accuracy_score(classesParaTesteNetflixBin, resultValidBinLR) * 100
print("{}{} ".format(acc3.round(2), "%"))

68.5% 
65.35% 
66.14% 


In [101]:
print (pd.crosstab(classesParaTesteNetflixBin, resultValidBin, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflixBin, resultValidBinSVM, rownames=['Real'], colnames=['Predito'], margins=True))

print (pd.crosstab(classesParaTesteNetflixBin, resultValidBinLR, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0   1  All
Real                
0        44  18   62
1        22  43   65
All      66  61  127
Predito   0   1  All
Real                
0        35  27   62
1        17  48   65
All      52  75  127
Predito   0   1  All
Real                
0        33  29   62
1        14  51   65
All      47  80  127


In [102]:
print(classification_report(classesParaTesteNetflixBin, resultValidBin, labels = [0,1]))

print(classification_report(classesParaTesteNetflixBin, resultValidBinSVM, labels = [0,1]))

print(classification_report(classesParaTesteNetflixBin, resultValidBinLR, labels = [0,1]))

             precision    recall  f1-score   support

          0       0.67      0.71      0.69        62
          1       0.70      0.66      0.68        65

avg / total       0.69      0.69      0.68       127

             precision    recall  f1-score   support

          0       0.67      0.56      0.61        62
          1       0.64      0.74      0.69        65

avg / total       0.66      0.65      0.65       127

             precision    recall  f1-score   support

          0       0.70      0.53      0.61        62
          1       0.64      0.78      0.70        65

avg / total       0.67      0.66      0.66       127




###### Obrigado por olhar o meu notebook! Fique livre para copiá-lo, altera-lo e utiliza-lo em seus estudos/projetos 